## Predicting Odontogenic Infection Research

In [76]:
%matplotlib notebook
import numpy as np
from numpy import percentile
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from scipy.stats import uniform, randint
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from statsmodels.nonparametric.kde import KDEUnivariate

In [364]:
import numpy as np
import pandas as pd

In [365]:
df = pd.read_excel("medata.xlsx")

In [366]:
df.head()

,ID,AGE,GENDER,TIME,DUR,DOD,TOOTH,DRUG,COMPLICATIONS,AGERAW,SYMP,SEVRYT
0,1,8,F,2,3,17,LW,MCX,NIL,50,6,2
1,2,10,F,2,2,10,LW,AMT,YES,60,10,2
2,3,6,F,3,2,9,LW,MCX,NIL,37,12,2
3,4,3,F,1,1,5,LW,MCT,YES,21,4,2
4,5,2,M,1,1,7,LW,MCX,YES,17,2,2


In [367]:
df = df.drop(['ID'], axis=1)

In [368]:
df.head()

,AGE,GENDER,TIME,DUR,DOD,TOOTH,DRUG,COMPLICATIONS,AGERAW,SYMP,SEVRYT
0,8,F,2,3,17,LW,MCX,NIL,50,6,2
1,10,F,2,2,10,LW,AMT,YES,60,10,2
2,6,F,3,2,9,LW,MCX,NIL,37,12,2
3,3,F,1,1,5,LW,MCT,YES,21,4,2
4,2,M,1,1,7,LW,MCX,YES,17,2,2


In [369]:
nominal_column = ['TOOTH','GENDER','DRUG','COMPLICATIONS']


In [370]:
dummy_df = pd.get_dummies(df[nominal_column])
df = pd.concat([df,dummy_df], axis=1)

In [371]:
df = df.drop(nominal_column, axis = 1)

In [372]:
df.head()

,AGE,TIME,DUR,DOD,AGERAW,SYMP,SEVRYT,TOOTH_LW,TOOTH_UP,GENDER_F,GENDER_M,DRUG_AMT,DRUG_MCT,DRUG_MCX,COMPLICATIONS_NIL,COMPLICATIONS_YES
0,8,2,3,17,50,6,2,1,0,1,0,0,0,1,1,0
1,10,2,2,10,60,10,2,1,0,1,0,1,0,0,0,1
2,6,3,2,9,37,12,2,1,0,1,0,0,0,1,1,0
3,3,1,1,5,21,4,2,1,0,1,0,0,1,0,0,1
4,2,1,1,7,17,2,2,1,0,0,1,0,0,1,0,1


In [373]:
df.columns

Index(['AGE', 'TIME', 'DUR', 'DOD', 'AGERAW', 'SYMP', 'SEVRYT', 'TOOTH_LW',
       'TOOTH_UP', 'GENDER_F', 'GENDER_M', 'DRUG_AMT', 'DRUG_MCT', 'DRUG_MCX',
       'COMPLICATIONS_NIL', 'COMPLICATIONS_YES'],
      dtype='object')

In [434]:
Features = df[['AGE', 'TIME', 'DUR', 'DOD', 'AGERAW', 'SYMP', 'SEVRYT', 'TOOTH_LW',
       'TOOTH_UP', 'GENDER_F', 'GENDER_M', 'DRUG_AMT', 'DRUG_MCT', 'DRUG_MCX',
       'COMPLICATIONS_NIL', 'COMPLICATIONS_YES']]
Target = df['COMPLICATIONS_YES']

In [435]:
X = Features
y = Target

In [436]:
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.75, test_size=0.25, random_state=0)

In [452]:
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.80, test_size=0.20, random_state=0)

clf = LogisticRegression(C=10).fit(X_train, y_train)
     

In [458]:
comp = clf.predict(X_test)

In [459]:
comp

array([0, 1, 0, 0, 0, 0, 1], dtype=uint8)

In [460]:
clf.score(X_train, y_train)

1.0

In [461]:
clf.score(X_test, y_test)

1.0

In [481]:
def display_scores(scores):
    print("Scores: {0}\nMean: {1:.3f}\nStd: {2:.3f}".format(scores, np.mean(scores), np.std(scores)))

In [485]:
from sklearn.model_selection import cross_val_score

scores_knn = cross_val_score(clf, X, y, scoring="accuracy", cv=3)

display_scores(scores_knn )

Scores: [1. 1. 1.]
Mean: 1.000
Std: 0.000


In [486]:
from scipy.stats import uniform, randint
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
import xgboost as xgb

#xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=0)
xgb_model =xgb.XGBClassifier(n_estimators=10, min_child_weight=5, max_depth=1) 

xgb_model.fit(X_train, y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=1, min_child_weight=5, missing=None,
       n_estimators=10, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [441]:
xgb_model.score(X_test, y_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.2857142857142857

In [444]:
from sklearn.svm import SVC
svm = SVC(kernel = 'rbf', C=1).fit(X_train, y_train)
svm_predicted_mc = svm.predict(X_test)
#svm.score(X_test, y_test)
svm_predicted_mc

array([1, 1, 1, 1, 1, 1, 1], dtype=uint8)

In [445]:
svm.score(X_test, y_test)

0.2857142857142857

In [446]:
y_test

11    0
22    1
10    0
2     0
16    0
14    0
28    1
Name: COMPLICATIONS_YES, dtype: uint8

In [469]:
from sklearn.dummy import DummyClassifier

# Negative class (0) is most frequent
constant = [0]
dummy_majority = DummyClassifier(strategy = 'most_frequent').fit(X_train, y_train)
# Therefore the dummy 'most_frequent' classifier always predicts class 0
y_dummy_predictions = dummy_majority.predict(X_test)

dummy_majority.score(X_test, y_test)

0.2857142857142857

In [470]:
from sklearn.ensemble import RandomForestClassifier
clf_= RandomForestClassifier() 
rfg = clf.fit(X_train, y_train)
predicted_mc = rfg.predict(X_test)

predicted_mc

array([0, 1, 0, 0, 0, 0, 1], dtype=uint8)

In [471]:
rfg.score(X_test, y_test)

1.0

In [478]:
from sklearn.neighbors import KNeighborsClassifier

knnreg = KNeighborsClassifier(n_neighbors = 5).fit(X_train, y_train)

TrainOutput_knn = knnreg.predict(X_train)
TestOutput_knn = knnreg.predict(X_test)


In [479]:
knnreg.score(X_test, y_test)

0.7142857142857143

In [ ]:
complication yes..predicting..